# Lecture 9
## Object Oriented Programming
### Monday, October 2nd 2017

In [1]:
from IPython.display import HTML

## Motiviation
We would like to find a way to represent complex, structured data in the context of our programming language.

For example, to represent a location, we might want to associate a `name`, a `latitude` and a `longitude` with it. 

Thus we would want to create a **compound data type** which carries this information.

In C, for example, this is a struct:

```C
struct location {
    float longitude;
    float latitude;
}
```

### REMEMBER: A language has 3 parts:

- expressions and statements: how to structure simple computations
- means of combination: how to structure complex computations
- means of abstraction: how to build complex units

## Review

- When we write a function, we give it some sensible name which can then be used by a "client" programmer. We don't care about how this function is implemented.  We just want to know its signature (API) and use it.

- In a similar way, we want to *encapsulate* our data: we dont want to know how it is stored and all that.  We just want to be able to use it. This is one of the key ideas behind object oriented programming. 

- To do this, write **constructors** that make objects.  We also write other functions that access or change data on the object. These functions are called the "methods" of the object, and are what the client programmer uses.

# First Examples

### Objects thru tuples:  An object for complex numbers

How might we implement such objects? First, lets think of tuples.

In [2]:
def Complex(a, b): # constructor
    return (a,b)

def real(c): # method
    return c[0]

def imag(c):
    return c[1]

def str_complex(c):
    return "{0}+{1}i".format(c[0], c[1])

In [3]:
c1 = Complex(1,2) # constructor
print(real(c1), "     ", str_complex(c1))

1       1+2i


But things aren't hidden so I can get through the interface:

In [4]:
c1[0]

1

Because I used a tuple, and a tuple is immutable, I can't change this complex number once it's created.

In [5]:
c1[0]=2

TypeError: 'tuple' object does not support item assignment

### Objects thru closures

Let's try an implementation that uses a closure to capture the value of arguments.

In [6]:
def Complex2(a, b): # constructor
    def dispatch(message): # capture a and b at constructor-run time
        if message=="real":
            return a
        elif message=='imag':
            return b
        elif message=="str":
            return "{0}+{1}i".format(a, b)
    return dispatch

In [7]:
z=Complex2(1,2)
print(z("real"), "     ", z("imag"), "     ", z("str"))

1       2       1+2i


This looks pretty good so far.

The only problem is that we don't have a way to change the real and imaginary parts.

For this, we need to add things called `setters`.

### Objects with Setters

In [8]:
def Complex3(a, b):
    in_a=a
    in_b=b
    def dispatch(message, value=None):
        nonlocal in_a, in_b
        if message=='set_real' and value != None:
            in_a = value
        elif message=='set_imag' and value != None:
            in_b = value
        elif message=="real":
            return in_a
        elif message=='imag':
            return in_b
        elif message=="str":
            return "{0}+{1}i".format(in_a, in_b)
    return dispatch

In [9]:
c3=Complex3(1,2)
print(c3("real"), "     ", c3("imag"), "     ", c3("str"))

1       2       1+2i


In [10]:
c3('set_real', 2)

In [11]:
print(c3("real"), "     ", c3("imag"), "     ", c3("str"))

2       2       2+2i


### Python Classes and instance variables

We constructed an object system above. But Python comes with its own.

Classes allow us to define our own *types* in the Python type system.

In [12]:
class ComplexClass():
    
    def __init__(self, a, b):
        self.real = a
        self.imaginary = b

`__init__` is a special method run automatically by Python.

It is a constructor.

`self` is the *instance* of the object.

It acts like `this` in `C++` but `self` is explicit.

In [13]:
HTML('<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=class%20ComplexClass%28%29%3A%0A%20%20%20%20%0A%20%20%20%20def%20__init__%28self,%20a,%20b%29%3A%0A%20%20%20%20%20%20%20%20self.real%20%3D%20a%0A%20%20%20%20%20%20%20%20self.imaginary%20%3D%20b%0A%0Ac1%20%3D%20ComplexClass%281,2%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=false&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>')

In [14]:
c1 = ComplexClass(1,2)
print(c1, c1.real)

<__main__.ComplexClass object at 0x10a96ca58> 1


In [15]:
print(vars(c1), "     ",type(c1))

{'real': 1, 'imaginary': 2}       <class '__main__.ComplexClass'>


In [16]:
c1.real=5.0
print(c1, "     ", c1.real, "     ", c1.imaginary)

<__main__.ComplexClass object at 0x10a96ca58>       5.0       2


## Inheritance and Polymorphism

### Inheritance

**Inheritance** is the idea that a "Cat" **is-a** "Animal" and a "Dog" **is-a** "Animal". 

Animals make sounds, but Cats Meow and Dogs Bark.

Inheritance makes sure that *methods not defined in a child are found and used from a parent*.

### Polymorphism

**Polymorphism** is the idea that an **interface** is specified, but not necessarily implemented, by a superclass and then the interface is implemented in subclasses (differently).

[Actually Polymorphism is much more complex and interesting than this, and this definition is really an outcome of polymorphism. But we'll come to this later.]

###  Example:  Super- and subclasses

In [17]:
class Animal():
    
    def __init__(self, name):
        self.name = name
        
    def make_sound(self):
        raise NotImplementedError
    
class Dog(Animal):
    
    def make_sound(self):
        return "Bark"
    
class Cat(Animal):
    
    def __init__(self, name):
        self.name = "A very interesting cat: {}".format(name)
        
    def make_sound(self):
        return "Meow"

* `Animal` is the superclass (a.k.a the base class).
* `Dog` and `Cat` are both subclasses (a.k.a derived classes) of the `Animal` superclass.

### Using the `Animal` class

In [18]:
a0 = Animal("David")
print(a0.name)
a0.make_sound()

David


NotImplementedError: 

In [19]:
a1 = Dog("Snoopy")
a2 = Cat("Hello Kitty")
animals = [a1, a2]
for a in animals:
    print(a.name)
    print(isinstance(a, Animal))
    print(a.make_sound())
    print('--------')

Snoopy
True
Bark
--------
A very interesting cat: Hello Kitty
True
Meow
--------


In [20]:
print(a1.make_sound, "     ", Dog.make_sound)

<bound method Dog.make_sound of <__main__.Dog object at 0x10aa64898>>       <function Dog.make_sound at 0x10aa48a60>


In [21]:
print(a1.make_sound())
print('----')
print(Dog.make_sound(a1))

Bark
----
Bark


In [22]:
Dog.make_sound()

TypeError: make_sound() missing 1 required positional argument: 'self'

### How does this all work?

In [26]:
HTML('<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=class%20Animal%28%29%3A%0A%20%20%20%20%0A%20%20%20%20def%20__init__%28self,%20name%29%3A%0A%20%20%20%20%20%20%20%20self.name%20%3D%20name%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20def%20make_sound%28self%29%3A%0A%20%20%20%20%20%20%20%20raise%20NotImplementedError%0A%20%20%20%20%0Aclass%20Dog%28Animal%29%3A%0A%20%20%20%20%0A%20%20%20%20def%20make_sound%28self%29%3A%0A%20%20%20%20%20%20%20%20return%20%22Bark%22%0A%20%20%20%20%0Aclass%20Cat%28Animal%29%3A%0A%20%20%20%20%0A%20%20%20%20def%20__init__%28self,%20name%29%3A%0A%20%20%20%20%20%20%20%20self.name%20%3D%20%22A%20very%20interesting%20cat%3A%20%7B%7D%22.format%28name%29%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20def%20make_sound%28self%29%3A%0A%20%20%20%20%20%20%20%20return%20%22Meow%22%0A%0Aa1%20%3D%20Dog%28%22Snoopy%22%29%0Aa2%20%3D%20Cat%28%22Hello%20Kitty%22%29%0Aanimals%20%3D%20%5Ba1,%20a2%5D%0Afor%20a%20in%20animals%3A%0A%20%20%20%20print%28a.name%29%0A%20%20%20%20print%28isinstance%28a,%20Animal%29%29%0A%20%20%20%20print%28a.make_sound%28%29%29%0A%20%20%20%20print%28\'--------\'%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=false&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>')

### Calling a superclasses initializer

* Say we dont want to do all the work of setting the name variable in the subclasses.

* We can set this "common" work up in the superclass and use `super` to call the superclasse's initializer from the subclas.

* See https://rhettinger.wordpress.com/2011/05/26/super-considered-super/

In [27]:
class Animal():
    
    def __init__(self, name):
        self.name=name
        print("Name is", self.name)
        
class Mouse(Animal):
    def __init__(self, name):
        self.animaltype="prey"
        super().__init__(name)
        print("Created %s as %s" % (self.name, self.animaltype))
    
class Cat(Animal):
    pass

a1 = Mouse("Tom")
print(vars(a1))
a2 = Cat("Jerry")
print(vars(a2))

Name is Tom
Created Tom as prey
{'animaltype': 'prey', 'name': 'Tom'}
Name is Jerry
{'name': 'Jerry'}


In [28]:
HTML('<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=class%20Animal%28%29%3A%0A%20%20%20%20%0A%20%20%20%20def%20__init__%28self,%20name%29%3A%0A%20%20%20%20%20%20%20%20self.name%3Dname%0A%20%20%20%20%20%20%20%20print%28%22Name%20is%22,%20self.name%29%0A%20%20%20%20%20%20%20%20%0Aclass%20Mouse%28Animal%29%3A%0A%20%20%20%20def%20__init__%28self,%20name%29%3A%0A%20%20%20%20%20%20%20%20self.animaltype%3D%22prey%22%0A%20%20%20%20%20%20%20%20super%28%29.__init__%28name%29%0A%20%20%20%20%20%20%20%20print%28%22Created%20%25s%20as%20%25s%22%20%25%20%28self.name,%20self.animaltype%29%29%0A%20%20%20%20%0Aclass%20Cat%28Animal%29%3A%0A%20%20%20%20pass%0A%0Aa1%20%3D%20Mouse%28%22Tom%22%29%0Aa2%20%3D%20Cat%28%22Jerry%22%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=false&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>')

## Interfaces

* The above examples show inheritance and polymorphism.
* Notice that we didn't actually need to set up the inheritance.
* We could have just defined 2 different classes and have them both `make_sound`.  The same code would work. 
* In `Java` and `C++` this is done more formally through Interfaces and  Abstract Base Classes, respectively plus inheritance.
* In Python, this agreement to define `make_sound` is called **duck typing**.

In [ ]:
#both implement the "Animal" Protocol, which consists of the one make_sound function
class Dog():
    
    def make_sound(self):
        return "Bark"
    
class Cat():
    
    def make_sound(self):
        return "Meow"  
    
a1 = Dog()
a2 = Cat()
animals = [a1, a2]
for a in animals:
    print(isinstance(a, Animal))
    print(a.make_sound())

## The Python Data Model

Duck typing is used throughout `Python`. Indeed it's what enables the "Python Data Model" 

- All python classes implicitly inherit from the root **object** class.
- The *Pythonic* way, is to just document your interface and implement it. 
- This usage of common **interfaces** is pervasive in *dunder* functions to comprise the `Python` data model.

####   `__repr__`  

The way printing works is that Python wants classes to implement a `__repr__` and a `__str__` method. It will use inheritance to give the built-in `object`s methods when these are not defined...but any class can define these. When an *instance* of such a class is interrogated with the `repr` or `str` function, then these underlying methods are called.

We'll see `__repr__` here. If you define `__repr__` you have made an object sensibly printable...

In [ ]:
class Animal():
    
    def __init__(self, name):
        self.name=name
        
    def __repr__(self):
        class_name = type(self).__name__
        return "Da %s(name=%r)" % (class_name, self.name)

In [ ]:
r = Animal("Rahul")
r

In [ ]:
print(r)

In [ ]:
repr(r)

### The pattern with dunder methods


**there are functions without double-underscores that cause the methods with the double-underscores to be called**

Thus `repr(an_object)` will cause `an_object.__repr__()` to be called. 

In user-level code, you *SHOULD NEVER* see the latter. In library level code, you might see the latter. The definition of the class is considered library level code.

#### Instance Equality via `__eq__`

Now we are in a position to answer the initial question: what makes two squirrels equal!

To do  this, we will add a new dunder method to the mix, the unimaginatively (thats a good thing) named `__eq__`.

In [ ]:
class Animal():
    
    def __init__(self, name):
        self.name=name
        
    def __repr__(self):
        class_name = type(self).__name__
        return "%s(name=%r)" % (class_name, self.name)
    
    def __eq__(self, other):
        return self.name==other.name # two animals are equal if there names are equal

In [ ]:
A=Animal("Tom")
B=Animal("Jane")
C=Animal("Tom")

Three separate object identities, but we made two of them equal!

In [ ]:
print(id(A), id(B), id(C))

print(A==B, B==C, A==C)

This is critical because it gives us a say in what equality means

### Python's power comes from the data model, composition, and delegation

The data model is used (from Fluent) to provide a:

>description of the interfaces of the building blocks of the language itself, such as sequences, iterators, functions, classes....

The special "dunder" methods we talk about are invoked by the python interpreter to beform basic operations. For example, `__getitem__` gets an item in a sequence. This is used to do something like `a[3]`. `__len__` is used to say how long a sequence is. Its invoked by the `len` built in function. 

A **sequence**, for example,  must implement `__len__` and `__getitem__`. Thats it.

The original reference for this data mode is: https://docs.python.org/3/reference/datamodel.html .

#### Tuple

An example of a sequence in Python is the tuple. This means, that it must support indexing and be able to tell us its length.

In [ ]:
a=(1,2)
a[0]

In [ ]:
len(a)

#### NamedTuples

One can use the `collections.namedtuple` "FACTORY" function to produces subclasses of tuples enhanced with field names and a classed name.

Consider, as an example (from Fluent Python):

In [ ]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])
type(Card)

In [ ]:
my_card = Card(rank='3', suit='diamond')
my_card, type(my_card)

In [ ]:
my_card.rank

#### A Custom Sequence

We now wish to create a `FrenchDeck` as an example of something that follows Python's Sequence protocol. Remember, the sequence protocol requires implementation of two methods: `__len__` and `__getitem__`. Thats it.

In [ ]:
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JKQA')
    suits="spade diamond club heart".split()
    
    def __init__(self):
        #composition: there are items IN this class that constutute its structure
        #delegation: the storage for this class is DELEGATED to this list below
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [ ]:
deck = FrenchDeck()
len(deck)

In [ ]:
deck[0], deck[-1], deck[3]

In [ ]:
deck[10:18]

Because we support the sequence protocol, you can use, in python, dunctions like `random.choice` DIRECTLY on instances of `FrenchDeck`. This is the power of interfaces and the data model.

In [ ]:
from random import choice
choice(deck)

### Building out our class: instances and classmethods

In [ ]:
class ComplexClass():
    def __init__(self, a, b):
        self.real = a
        self.imaginary = b
        
    @classmethod
    def make_complex(cls, a, b):
        return cls(a, b)
        
    def __repr__(self):
        class_name = type(self).__name__
        return "%s(real=%r, imaginary=%r)" % (class_name, self.real, self.imaginary)
        
    def __eq__(self, other):
        return (self.real == other.real) and (self.imaginary == other.imaginary)

In [ ]:
c1 = ComplexClass(1,2)
c1

`make_complex` is a class method. See how its signature is different above. It is a factory to produce instances.

In [ ]:
c2 = ComplexClass.make_complex(1,2)
c2

In [ ]:
c1 == c2

You can see where we are going with this. Wouldnt it be great to define adds, subtracts, etc? Later...

### Static Methods, Class Methods, Instance Methods

What's really going on under the hood here?

In [ ]:
#from fluent python
class Demo():
    @classmethod
    def klassmeth(*args): #class methods do not have to return an instance of the class
        return args
    
    @staticmethod
    def statmeth(*args): #this is just a regular function
        return args
    
    def instmeth(*args): #this is a true blue instance method
        return args
    

In [ ]:
notademo = Demo.statmeth(1,2)
print(type(notademo))
notademo

In [ ]:
ademo = Demo.klassmeth(1,2)
print(type(ademo))
ademo

In [ ]:
ademo = Demo()
Demo.instmeth(ademo, 1,2)

In [ ]:
ademo.instmeth(1,2)

In [ ]:
HTML('<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=%23from+fluent+python%0Aclass+Demo(%29%3A%0A++++%40classmethod%0A++++def+klassmeth(*args%29%3A%0A++++++++return+args%0A++++%0A++++%40staticmethod%0A++++def+statmeth(*args%29%3A%0A++++++++return+args%0A++++%0A++++def+instmeth(*args%29%3A%0A++++++++return+args%0A++++%0Aprint(Demo.statmeth(1,2%29%29%0Aprint(Demo.klassmeth(1,2%29%29%0Aademo+%3D+Demo(%29%0Aprint(Demo.instmeth(ademo,+1,2%29%29%0Aprint(ademo.instmeth(1,2%29%29&origin=opt-frontend.js&cumulative=false&heapPrimitives=false&textReferences=false&py=3&rawInputLstJSON=%5B%5D&curInstr=0&codeDivWidth=350&codeDivHeight=400"> </iframe>')

### Class variables and instance variables



In [ ]:
class Demo2():
    classvar=1
      
ademo2 = Demo2()
print(Demo2.classvar, ademo2.classvar)
ademo2.classvar=2 #different from the classvar above
print(Demo2.classvar, ademo2.classvar)

In [ ]:
HTML('<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=class+Demo2(%29%3A%0A++++classvar%3D1%0A++++++%0Aademo2+%3D+Demo2(%29%0Aprint(Demo2.classvar,+ademo2.classvar%29%0Aademo2.classvar%3D2%0Aprint(Demo2.classvar,+ademo2.classvar%29&origin=opt-frontend.js&cumulative=false&heapPrimitives=false&textReferences=false&py=3&rawInputLstJSON=%5B%5D&curInstr=0&codeDivWidth=350&codeDivHeight=400"> </iframe>')

## Code and Data for objects

Lets give ourselves a very vague idea of how objects work in Python, from both a storage (data) and running (code) perspective. We'll expand on this later.

In [ ]:
class A(object):
    
    def __init__(self, x):
        self.x = x
        
    def doit(self, y):
        return self.x + y

In [ ]:
#from https://bitbucket.org/yaniv_aknin/pynards/src/c4b61c7a1798766affb49bfba86e485012af6d16/common/blog.py?at=default&fileviewer=file-view-default
import dis
import types

def get_code_object(obj, compilation_mode="exec"):
    if isinstance(obj, types.CodeType):
        return obj
    elif isinstance(obj, types.FrameType):
        return obj.f_code
    elif isinstance(obj, types.FunctionType):
        return obj.__code__
    elif isinstance(obj, str):
        try:
            return compile(obj, "<string>", compilation_mode)
        except SyntaxError as error:
            raise ValueError("syntax error in passed string") from error
    else:
        raise TypeError("get_code_object() can not handle '%s' objects" %
                        (type(obj).__name__,))

def diss(obj, mode="exec", recurse=False):
    _visit(obj, dis.dis, mode, recurse)

def ssc(obj, mode="exec", recurse=False):
    _visit(obj, dis.show_code, mode, recurse)

def _visit(obj, visitor, mode="exec", recurse=False):
    obj = get_code_object(obj, mode)
    visitor(obj)
    if recurse:
        for constant in obj.co_consts:
            if type(constant) is type(obj):
                print()
                print('recursing into %r:' % (constant,))
                _visit(constant, visitor, mode, recurse)

Notice below how an unbound object is `LOAD_FAST`ed in `__init__`.

In [ ]:
dis.dis(A)

In [ ]:
def f():
    a=A(5)
    a.doit(3)

Notice here that both the constructor and the method are called as functions. In the former, an implicit unbound object is used. 

In [ ]:
diss(f)

`dir` for classes contains the names of its attributes, and recursively of the attributes of its bases. `var` on an object gets the contents of a special attribute called `__dict__`.

In [ ]:
dir(A), vars(A)

In [ ]:
a=A(5)

In [ ]:
dir(a), vars(a)

There is some kind of a table implementation for python objects (its written in C). This implementations allows us to look for attributes and methods, and if not found look elsewhere. The exact details are complex, using descriptors and other lookups, and we'll tackle them in more detail later. But currently it suffices us to know that lookup first happens in the instance table, followed by the class table (methods) and fif not there somewhere up in the inheritance hierarchy.

In [ ]:
A.__class__, a.__class__